# 1. Get list of players and teams ID from FotMob

In [1]:
import requests
import pandas as pd
import json

# get JSON response
url = "https://www.fotmob.com/api/leagueseasondeepstats?id=47&season=17664&stat=rating&type=players"
response = requests.get(url)
json_data = response.text

# convert into JSON and get node statsData
data_dict = json.loads(json_data)
df_players = pd.DataFrame(data_dict['statsData'])

# print the resulting DataFrame
print(df_players)
df_players.to_csv('fotmob-premier-league-players-list.csv', index=False)

          id  teamId                                nameAndSubstatValue  \
0     675088    8456               {'name': 'Rodri', 'substatValue': 5}   
1     191869   10261     {'name': 'Kieran Trippier', 'substatValue': 3}   
2     961995    9825         {'name': 'Bukayo Saka', 'substatValue': 2}   
3     434325    9825       {'name': 'Thomas Partey', 'substatValue': 2}   
4     534670    9825     {'name': 'Martin Ødegaard', 'substatValue': 6}   
..       ...     ...                                                ...   
280   819762    8678        {'name': 'Chris Mepham', 'substatValue': 0}   
281   158545    8654     {'name': 'Michail Antonio', 'substatValue': 0}   
282   706297    9826  {'name': 'Jean-Philippe Mateta', 'substatValue...   
283  1177312    8466          {'name': 'Sékou Mara', 'substatValue': 0}   
284    19243    8602         {'name': 'Diego Costa', 'substatValue': 0}   

    statValue  rank     type statFormat substatFormat  
0        7.90     1  players   fraction    

# 2. Download players information from player page https://www.fotmob.com/_next/data/3696/players/961995/x-x.json

https://www.fotmob.com/api/playerData?id=576165


In [18]:
def get_player_data_from_fotmob(player_id):
  url = "https://www.fotmob.com/api/playerData?id=" + str(player_id)
  response = requests.get(url)
  json_data = response.text

  # convert into JSON and get node statsData
  data_dict = json.loads(json_data)
  _player_height = None
  _player_preferred_foot = None
  _player_age = None
  _player_country = None
  _player_shirt = None
  _player_market_value = None
  if 'playerProps' in data_dict:
    for playerProp in data_dict['playerProps']:
      if playerProp['title'] == 'Height':
        _player_height = playerProp['value']
      elif playerProp['title'] == 'Preferred foot':
        _player_preferred_foot = playerProp['value']
      elif playerProp['title'] == 'Age':
        _player_age = playerProp['value']
      elif playerProp['title'] == 'Country':
        _player_country = playerProp['value']
      elif playerProp['title'] == 'Shirt':
        _player_shirt = playerProp['value']
      elif playerProp['title'] == 'Market value':
        _player_market_value = playerProp['value']

  # player information
  player_data = {
    'player_id' : data_dict['id'],
    'player_name' : data_dict['name'],
    'team_id' : data_dict['origin']['teamId'],
    'team_color' : data_dict['origin']['teamColor'],
    'team' : data_dict['origin']['teamName'],
    'player_position_primary' : data_dict['origin']['positionDesc']['primaryPosition'],
    'player_position_nonprimary' : data_dict['origin']['positionDesc']['nonPrimaryPositions'],
    'player_height' : _player_height,
    'player_preferred_foot' : _player_preferred_foot,
    'player_age' : _player_age,
    'player_country' : _player_country,
    'player_shirt' : _player_shirt,
    'player_market_value' : _player_market_value,
    'player_recent_matches' : data_dict['recentMatches']['All competitions'],
  }

  # player matches
  player_matches_data = []
  for _match in data_dict['recentMatches']['All competitions']:
    player_matches_data.append({
        'player_id' : player_data['player_id'],
        'match_date' : _match['date']['utcTime'],
        'match_id' : _match['versus']['matchId'],
        'match_home_team' : _match['htName'],
        'match_away_team' : _match['atName'],
        'match_home_team_score' : _match['versus']['homeTeamScore'],
        'match_away_team_score' : _match['versus']['awayTeamScore'],
        'match_player_minutes_played' : _match['minutesPlayed'],
        'match_player_goals' : _match['goals'],
        'match_player_assists' : _match['assists'],
        'match_player_yellow_cards' : _match['yellowCards'],
        'match_player_red_cards' : _match['redCards'],
        'match_player_rating' : _match['ratingProps']['num'],
        'match_player_rating_color' : _match['ratingProps']['bgcolor'],
    })
    
  # print the resulting DataFrame
  return player_data, player_matches_data

player_data, player_matches_data = get_player_data_from_fotmob(961995)
print(player_data)

{'player_id': 961995, 'player_name': 'Bukayo Saka', 'team_id': 9825, 'team_color': '#bd0510', 'team': 'Arsenal', 'player_position_primary': 'Right Winger', 'player_position_nonprimary': 'Left Winger', 'player_height': '178 cm', 'player_preferred_foot': 'Left', 'player_age': 21, 'player_country': 'England', 'player_shirt': 7, 'player_market_value': '€105M', 'player_recent_matches': [{'htName': 'Arsenal', 'atName': 'Sporting CP', 'date': {'utcTime': '2023-03-16T21:00:00.000Z', 'timezone': None}, 'versus': {'matchId': 4128638, 'opponentName': 'Sporting CP', 'opponentId': 9768, 'homeTeamScore': 4, 'awayTeamScore': 6, 'highLightHomeTeam': True}, 'minutesPlayed': 55, 'goals': 0, 'assists': 0, 'yellowCards': 0, 'redCards': 0, 'ratingProps': {'num': '6.5', 'bgcolor': '#f08022'}}, {'htName': 'Fulham', 'atName': 'Arsenal', 'date': {'utcTime': '2023-03-12T15:00:00.000Z', 'timezone': None}, 'versus': {'matchId': 3901195, 'opponentName': 'Fulham', 'opponentId': 9879, 'homeTeamScore': 0, 'awayTeamSc

In [41]:
data_players = []
data_player_matches = []
for index, row in df_players.iterrows():
  player_data, player_matches_data = get_player_data_from_fotmob(row['id'])
  data_players.append(player_data)
  data_player_matches += player_matches_data

In [42]:
# print the resulting DataFrame
df_data_players = pd.DataFrame(data_players)
if 'player_recent_matches' in df_data_players:
  df_data_players = df_data_players.drop(columns=['player_recent_matches'])
df_data_players.to_csv('fotmob-premier-league-players.csv', index=False)

# print the resulting DataFrame
df_data_player_matches = pd.DataFrame(data_player_matches)
df_data_player_matches.to_csv('fotmob-premier-league-players-matches.csv', index=False)



> Bloc en retrait

